In [7]:
# !pip install -U torchtext==0.8.0
# !pip install -U torchtext==0.6.0 # This worked

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

In [ ]:
######### Loading from JSON/CSV/TSV files #########

# STEPS:
# 1. Specify how preprocessing should be done -> Fields
# 2. Use Dataset to load the data -> TabularDataset (JSON/CSV/TSV Files)
# 3. Construct an iterator to do batching & padding -> BucketIterator

In [10]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [13]:
import os
import zipfile
import numpy as np

dir = "/content/drive/My Drive/Colab Notebooks/PyTorch/Aladdin Pearson/Data/"
files = os.listdir(dir)
files

['train.csv', 'test.csv', 'test.json', 'train.json']

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [15]:
# python -m spacy download en
spacy_en = spacy.load("en")

In [19]:
tokenize = lambda x: x.split()

# def tokenize(text):
#     return [tok.text for tok in spacy_en.tokenizer(text)]

In [20]:
quote = Field(sequential=True, use_vocab=True, tokenize=tokenize, lower=True)
score = Field(sequential=False, use_vocab=False)

fields = {"quote": ("q", quote), "score": ("s", score)} 
# When want to get the batches we will use batch.q

train_data, test_data = TabularDataset.splits(
                    path= dir, 
                    train="train.json", 
                    test="test.json",
                    format="json", 
                    # validation = "validation.json" 
                    fields=fields
)

# # train_data, test_data = TabularDataset.splits(
# #                                         path='mydata',
# #                                         train='train.csv',
# #                                         test='test.csv',
# #                                         format='csv',
# #                                         fields=fields)

quote.build_vocab(train_data, max_size=10000, min_freq=1, vectors="glove.6B.100d")
# min_freq=2 of atleast 2

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  self.sequential = sequential
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:13: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  ex = cls()
.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 399999/400000 [00:19<00:00, 20420.76it/s]


In [21]:
train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), batch_size=2, device=device
)

# Iterator to do the batching and the padding

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  sort_within_batch=None):


In [23]:
for batch in train_iterator:
  print(batch.q)
  print(batch.s)

tensor([[10, 27],
        [21, 29],
        [ 4,  7],
        [ 3, 26],
        [ 6, 18],
        [11,  2],
        [17, 25],
        [ 4,  1],
        [ 3,  1],
        [30,  1],
        [28,  1],
        [ 5,  1],
        [13,  1],
        [ 2,  1],
        [ 9,  1],
        [23,  1]])
tensor([1, 0])
tensor([[33],
        [19],
        [24],
        [14],
        [15],
        [34],
        [32],
        [31],
        [16],
        [20],
        [22],
        [12],
        [ 5],
        [ 8]])
tensor([1])


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  self.batch_size = len(data)


In [25]:
######### Training a simple LSTM on this toy data of ours #########
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
        self.fc_out = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(1), self.hidden_size).to(device)

        embedded = self.embedding(x)
        outputs, _ = self.lstm(embedded, (h0, c0))
        prediction = self.fc_out(outputs[-1, :, :])

        return prediction




In [26]:
# Hyperparameters
input_size = len(quote.vocab)
hidden_size = 512
num_layers = 2
embedding_size = 100
learning_rate = 0.005
num_epochs = 10

In [27]:
# Initialize network
model = RNN_LSTM(input_size, embedding_size, hidden_size, num_layers).to(device)

# (NOT COVERED IN YOUTUBE VIDEO): Load the pretrained embeddings onto our model
pretrained_embeddings = quote.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2709,  0.0440, -0.0203,  ..., -0.4923,  0.6369,  0.2364],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.4989,  0.7660,  0.8975,  ..., -0.4118,  0.4054,  0.7850],
        [-0.5718,  0.0463,  0.8673,  ..., -0.3566,  0.9293,  0.8995]])

In [28]:
# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, batch in enumerate(train_iterator):
        # Get data to cuda if possible
        data = batch.q.to(device=device)
        targets = batch.s.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores.squeeze(1), targets.type_as(scores))

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  self.batch_size = len(data)
